1\. **PCA on 3D dataset**

* Generate a dataset with 3 features each with N entries (N being ${\cal O}(1000)$). With $N(\mu,\sigma)$ the normal distribution with mean $\mu$ and $\sigma$  standard deviation, generate the 3 variables $x_{1,2,3}$ such that:
    * $x_1$ is distributed as $N(0,1)$
    * $x_2$ is distributed as $x_1+N(0,3)$
    * $x_3$ is given by $2x_1+x_2$
* Find the eigenvectors and eigenvalues of the covariance matrix of the dataset
* Find the eigenvectors and eigenvalues using SVD. Check that the two procedures yield to same result
* What percent of the total dataset's variability is explained by the principal components? Given how the dataset was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of the total variability is retained.
* Redefine the data in the basis yielded by the PCA procedure
* Plot the data points in the original and the new coordiantes as a set of scatter plots. Your final figure should have 2 rows of 3 plots each, where the columns show the (0,1), (0,2) and (1,2) proejctions.


In [1]:
import numpy as np
import numpy.random as npr
import pandas as pd
from scipy import linalg as la

import matplotlib.pyplot as plt
%matplotlib inline

In [62]:
N = 2000
X = np.zeros((N,3))
X[:,0] = npr.normal(0,1, N)
X[:,1] = X[:,0] + npr.normal(0,3,N)
X[:,2] = 2*X[:,0] + X[:,1]

np.set_printoptions(suppress=True)

Cov = np.cov(X.T)
print("Covariance Matrix", '\n', Cov,'\n')
l, V = la.eig(Cov)
print("Eigenvalues with eig",'\n',np.real_if_close(l),'\n')
print(V,'\n')

U, spectrum, Vt = la.svd(Cov)
print("shapes:", U.shape,  spectrum.shape, Vt.shape, '\n')

print ("Spectrum", '\n',np.real_if_close(spectrum),'\n')
print ("U \n", U,'\n')
print ("Vt \n", Vt,'\n')

# Let's verify the definition of SVD by hand
D = np.zeros((3,3))
for i in range(3):
    D[i, i] = spectrum[i]
SVD = np.dot(U, np.dot(D, Vt))
print ("Covariance matrix reconstructed from SVD by definition \n", SVD, '\n')
np.allclose(SVD, Cov)

X0 = [X[:,i] - np.mean(X[:,i]) for i in range (0,3)]
U0, spectrum0, Vt0 = np.linalg.svd(X0)

l_svd = spectrum0**2/(N-1)
V_svd = U0
print("Eigenvalues with svd",'\n',l_svd, '\n')
print(V_svd, '\n')

Covariance Matrix 
 [[ 0.98116436  1.12855974  3.09088845]
 [ 1.12855974 10.1209376  12.37805708]
 [ 3.09088845 12.37805708 18.55983399]] 

Eigenvalues with eig 
 [27.79313115 -0.          1.8688048 ] 

[[-0.11755115 -0.81649658  0.56525663]
 [-0.57474401 -0.40824829 -0.7092268 ]
 [-0.80984631  0.40824829  0.42128646]] 

shapes: (3, 3) (3,) (3, 3) 

Spectrum 
 [27.79313115  1.8688048   0.        ] 

U 
 [[-0.11755115  0.56525663 -0.81649658]
 [-0.57474401 -0.7092268  -0.40824829]
 [-0.80984631  0.42128646  0.40824829]] 

Vt 
 [[-0.11755115 -0.57474401 -0.80984631]
 [ 0.56525663 -0.7092268   0.42128646]
 [ 0.81649658  0.40824829 -0.40824829]] 

Covariance matrix reconstructed from SVD by definition 
 [[ 0.98116436  1.12855974  3.09088845]
 [ 1.12855974 10.1209376  12.37805708]
 [ 3.09088845 12.37805708 18.55983399]] 

Eigenvalues with svd 
 [27.79313115  1.8688048   0.        ] 

[[-0.11755115  0.56525663 -0.81649658]
 [-0.57474401 -0.7092268  -0.40824829]
 [-0.80984631  0.42128646  0.4

In [75]:
Lambda=np.diag(l)
print (np.real_if_close(Lambda))
print ("A.trace():", Cov.trace())
print ("Lambda.trace():", np.real_if_close(Lambda.trace()))

print ("1st eigenvalue weight \n", np.real_if_close(Lambda[0,0])/np.real_if_close(Lambda.trace()))
print ("2nd eigenvalue weight \n", np.real_if_close(Lambda[1,1])/np.real_if_close(Lambda.trace()))
print ("3rd eigenvalue weight \n", np.real_if_close(Lambda[2,2])/np.real_if_close(Lambda.trace()))

[[27.79313115  0.          0.        ]
 [ 0.         -0.          0.        ]
 [ 0.          0.          1.8688048 ]]
A.trace(): 29.661935948472134
Lambda.trace(): 29.66193594847215
1st eigenvalue weight 
 0.9369965331316857
2nd eigenvalue weight 
 -7.196162477714215e-18
3rd eigenvalue weight 
 0.0630034668683142


2\. **PCA on a nD dataset**

Start from the dataset you have genereted in the previous exercise and add uncorrelated random noise. Such noise should be represented by other 10 uncorrelated variables normal distributed, with standar deviation much smaller (say, a factor 50) than those used to generate the $x_1$ and $x_2$.

Repeat the PCA procedure and compare the results with what you obtained before

3 \. **Looking at an oscillating spring** (optional)

Imagine you have $n$ cameras looking at a spring oscillating along the $x$ axis. Each  camera record the motion of the spring looking at it along a given direction defined by the pair $(\theta_i, \phi_i)$, the angles in spherical coordinates. 

Start from the simulation of the records (say ${\cal O}(1000)$) of the spring's motion along the x axis, assuming a little random noise affects the measurements along the $y$. Rotate such dataset to emulate the records of each camera.

Perform a Principal Component Analysis on the thus obtained dataset, aiming at finding the only one coordinate that really matters.


4\. **PCA on the MAGIC dataset** (optional)

Perform a PCA on the magic04.data dataset

In [ ]:
# get the dataset and its description on the proper data directory
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data -P ~/data/
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.names -P ~/data/ 